## Preliminaries

#### Purpose

The purpose of this notebook is to demonstrate the data files that are generated and used for the SPY weekly backtest that I am conducting for Parallax Asset Management.

#### Loading Modules

In [1]:
import numpy as np
import pandas as pd
import numexpr

#### Changing Display Settings

Thi code changes some of the default display settings so as to suit my workflow a bit more.

In [2]:
# setting maximum number of rows to display
pd.options.display.max_rows = 6

# displays all output from code cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## The Data

I'll now load the data files one-by-one and give a brief description of each.  I will refer to each file by the name of it's data-frame representation.  So, the "chain description" file is referred to as `df_chain_desc`, which is the variable name of the dataframe that the CSV file is read into.

#### Chain Description

The `df_chain` file is the smallest and most coarse of all the data files.  It contains each "chain" that is traded in the strategy.  I define a chain as an underlying/expiration combination.  So the first chain in `df_chain` is the 12/27/2013 expiration of SPY.  Since the weekly strategy we are backtesting in only for SPY, all the chains in `df_chain` will have the same underlying.

In [3]:
df_chain_desc = pd.read_csv("spy_weekly_chain_desc_5yr.csv")
df_chain_desc

underlying  monthly  expiration last_trade_date   execution  d2x  \
0          SPY    False  2013-12-27      2013-12-27  2013-12-20    4   
1          SPY    False  2014-01-03      2014-01-03  2013-12-27    4   
2          SPY    False  2014-01-10      2014-01-10  2014-01-03    5   
..         ...      ...         ...             ...         ...  ...   
253        SPY     True  2018-11-16      2018-11-16  2018-11-09    5   
254        SPY    False  2018-11-23      2018-11-23  2018-11-16    4   
255        SPY    False  2018-11-30      2018-11-30  2018-11-23    5   

     num_opts  exec_day_volume  
0          22           516575  
1          20           198107  
2          24           330390  
..        ...              ...  
253        51           857561  
254        51           368356  
255        64           212192  

[256 rows x 8 columns]

The columns of note are:
    
          execution: the date the options are traded for this chain
                d2x: number of days to expiration from the moment the options are traded
           num_opts: number of tradeable OTM options on execution day
    exec_day_volume: the volume for this chain on execution day

#### Chain History

The `df_chain_hist` file contains one row for every trading day for each chain.  So each chain will have 4 or 5 rows in `df_chain_hist`.  Probably the most useful column are the variance swap-rates, which is my preferred method for measuring market implied-volatility.  These can be used to determine a sell/no-sell signal based on comparison to some kind of estimate of realized vol.

In [4]:
df_chain_hist = pd.read_csv("spy_weekly_chain_hist_5yr.csv")
df_chain_hist

underlying  expiration  trade_date last_trade_date  implied_forward  \
0           SPY  2013-12-27  2013-12-20      2013-12-27       181.700000   
1           SPY  2013-12-27  2013-12-23      2013-12-27       182.475000   
2           SPY  2013-12-27  2013-12-24      2013-12-27       183.082000   
...         ...         ...         ...             ...              ...   
1489        SPY  2018-11-30  2018-11-28      2018-11-30       274.217000   
1490        SPY  2018-11-30  2018-11-29      2018-11-30       274.106000   
1491        SPY  2018-11-30  2018-11-30      2018-11-30       275.684998   

      bid_swap_rate  ask_swap_rate  mid_swap_rate  
0          0.105034       0.107660       0.106355  
1          0.121282       0.124316       0.122808  
2          0.148540       0.152255       0.150409  
...             ...            ...            ...  
1489       0.320543       0.325755       0.323159  
1490       0.453316       0.460687       0.457016  
1491       0.000000       0.000000       0.000000  

[1492 rows x 8 columns]

The columns of note are:
    
    implied_forward: forward price implied by option prices, calculated using put-call parity
      bid_swap_rate: variance swap rate calculated from option bid

#### Option History

`df_opt_hist` is used to determine two separate parts of the backtest: 1) the trades for each chain; 2) the pnl on each trade.  Towards this end, for the execution date of each chain, all tradeable OTM options are listed (one row for each option).  For each trade-date after the execution date, there is a row for each otm-option's settle price - these are what are used to calculate the PNL on the trades.

In [5]:
df_opt_hist = pd.read_csv("spy_weekly_opt_hist_5yr.csv")
df_opt_hist

underlying_symbol  underlying_price  type  expiration   data_date  \
0                   SPY        181.610001   put  2013-12-27  2013-12-20   
1                   SPY        181.610001   put  2013-12-27  2013-12-20   
2                   SPY        181.610001   put  2013-12-27  2013-12-20   
...                 ...               ...   ...         ...         ...   
48617               SPY        275.684998  call  2018-11-30  2018-11-30   
48618               SPY        275.684998  call  2018-11-30  2018-11-30   
48619               SPY        275.684998  call  2018-11-30  2018-11-30   

       strike       bid       ask       mid   volume  open_interest  \
0       175.5  0.110000  0.120000  0.115000   2272.0         2994.0   
1       176.0  0.120000  0.130000  0.125000  10209.0        12032.0   
2       176.5  0.130000  0.140000  0.135000   4976.0        15279.0   
...       ...       ...       ...       ...      ...            ...   
48617   275.0  0.684998  0.684998  0.684998  80221.0        28535.0   
48618   275.5  0.184998  0.184998  0.184998  30540.0        14612.0   
48619   276.0  0.000000  0.000000  0.000000  64814.0        27031.0   

       t1_open_interest  iv_mean  implied_vol     delta      vega     theta  
0                4171.0   0.1719     0.181563  0.063110  0.028368 -0.064381  
1               14810.0   0.1638     0.173209  0.070581  0.030933 -0.066972  
2               14875.0   0.1553     0.164273  0.078784  0.033640 -0.069077  
...                 ...      ...          ...       ...       ...       ...  
48617               0.0   0.3000     0.000000  1.000000  0.000000  0.000000  
48618               0.0   0.3000     0.000000  1.000000  0.000000  0.000000  
48619               0.0   0.3000     0.000000  0.000000  0.000000  0.000000  

[48620 rows x 17 columns]

#### Trade Master

The trade-master dataframe, `df_trade_master`, consists of all the trades on for each chain.  As a relic of previous research, there are three different deltas of trades listed: 10%, 30%, 50%.  These are demarcated by the `variation` column.

In [6]:
df_trade_master = pd.read_csv("spy_weekly_trade_master_5yr.csv")
df_trade_master

variation  type  expiration underlying   execution last_trade_date  \
0           0.1  call  2013-12-27        SPY  2013-12-20      2013-12-27   
1           0.1  call  2014-01-03        SPY  2013-12-27      2014-01-03   
2           0.1  call  2014-01-10        SPY  2014-01-03      2014-01-10   
...         ...   ...         ...        ...         ...             ...   
1533        0.5   put  2018-11-16        SPY  2018-11-09      2018-11-16   
1534        0.5   put  2018-11-23        SPY  2018-11-16      2018-11-23   
1535        0.5   put  2018-11-30        SPY  2018-11-23      2018-11-30   

     underlying_symbol  underlying_price type1 expiration1    ...      ask  \
0                  SPY        181.610001  call  2013-12-27    ...     0.11   
1                  SPY        183.845001  call  2014-01-03    ...     0.11   
2                  SPY        182.884995  call  2014-01-10    ...     0.11   
...                ...               ...   ...         ...    ...      ...   
1533               SPY        277.820007   put  2018-11-16    ...     2.16   
1534               SPY        273.730011   put  2018-11-23    ...     2.15   
1535               SPY        263.250000   put  2018-11-30    ...     2.93   

        mid  volume  open_interest  t1_open_interest  iv_mean  implied_vol  \
0     0.100     463           2424              2671   0.0917     0.093859   
1     0.100    1823            255              1947   0.0852     0.087870   
2     0.105   29002          22722             15395   0.0975     0.091043   
...     ...     ...            ...               ...      ...          ...   
1533  2.135   10469           7304              9881   0.1608     0.144631   
1534  2.140    4212           3826              4600   0.1432     0.158470   
1535  2.905    4122           5728              6793   0.2228     0.198666   

         delta      vega     theta  
0     0.098994  0.039879 -0.046788  
1     0.103453  0.041679 -0.045779  
2     0.095872  0.043838 -0.039911  
...        ...       ...       ...  
1533  0.478524  0.155853 -0.225412  
1534  0.489748  0.137716 -0.272797  
1535  0.491004  0.147790 -0.293608  

[1536 rows x 23 columns]

#### PNL Master

`df_pnl_master` contains the price-history, and hence the pnls for the unscaled trades in `df_trade_master`.  Noteworthy columns are:

    dly_opt_pnl: daily pnl for option only
     dly_dh_pnl: daily pnl for delta-hedge only
    dly_tot_pnl: dly_opt_pnl + dly_dh_pnl

In [7]:
df_pnl_master = pd.read_csv("spy_weekly_pnl_master_5yr.csv")
df_pnl_master

underlying_symbol  underlying_price  type  expiration   data_date  \
0                  SPY        181.610001  call  2013-12-27  2013-12-20   
1                  SPY        182.529999  call  2013-12-27  2013-12-23   
2                  SPY        182.929993  call  2013-12-27  2013-12-24   
...                ...               ...   ...         ...         ...   
8949               SPY        274.359985   put  2018-11-30  2018-11-28   
8950               SPY        273.950012   put  2018-11-30  2018-11-29   
8951               SPY        275.684998   put  2018-11-30  2018-11-30   

      strike   bid   ask    mid  volume       ...         implied_vol  \
0      184.5  0.09  0.11  0.100   463.0       ...            0.093859   
1      184.5  0.07  0.08  0.075  1781.0       ...            0.078971   
2      184.5  0.07  0.10  0.085  1496.0       ...            0.077759   
...      ...   ...   ...    ...     ...       ...                 ...   
8949   263.0  0.05  0.06  0.055  2860.0       ...            0.239881   
8950   263.0  0.01  0.02  0.015  3717.0       ...            0.279565   
8951   263.0  0.00  0.00  0.000    37.0       ...            0.000000   

         delta      vega     theta  variation  dly_opt_pnl  dly_dh_pnl  \
0     0.098994  0.039879 -0.046788        0.1        -0.02    0.000000   
1     0.100884  0.035166 -0.046285        0.1         0.03    0.091074   
2     0.133438  0.035131 -0.068293        0.1        -0.02    0.040353   
...        ...       ...       ...        ...          ...         ...   
8949  0.024703  0.014135 -0.084771        0.5         0.46   -1.019929   
8950  0.009202  0.004279 -0.059818        0.5         0.04    0.010128   
8951  0.000000  0.000000  0.000000        0.5         0.02   -0.015965   

      dly_opt_mid_pnl  dly_tot_pnl  dly_tot_mid_pnl  
0               0.000    -0.020000         0.000000  
1               0.025     0.121074         0.116074  
2              -0.010     0.020353         0.030353  
...               ...          ...              ...  
8949            0.455    -0.559929        -0.564929  
8950            0.040     0.050128         0.050128  
8951            0.015     0.004035        -0.000965  

[8952 rows x 23 columns]

#### Position Scaling

`df_position_scaling` is where I store the scaling factors to make all the trades the same size.  My basic rule of thumb is to sell a little under \$0.05 of premium per day.  So for an expiration that has 4 days to expiration, I sell about \$0.20 of premium; and for an expiration that has 5 days to expiration, I sell about \$0.25 of premium.

In [8]:
df_position_scaling = pd.read_csv("spy_weekly_position_scaling_5yr.csv")
df_position_scaling

variation underlying  expiration   execution  d2x  bid.put  ask.put  \
0          0.1        SPY  2013-12-27  2013-12-20    4     0.15     0.17   
1          0.3        SPY  2013-12-27  2013-12-20    4     0.45     0.47   
2          0.5        SPY  2013-12-27  2013-12-20    4     0.76     0.78   
..         ...        ...         ...         ...  ...      ...      ...   
765        0.1        SPY  2018-11-30  2018-11-23    5     0.42     0.44   
766        0.3        SPY  2018-11-30  2018-11-23    5     1.50     1.54   
767        0.5        SPY  2018-11-30  2018-11-23    5     2.88     2.93   

     mid.put  bid.call  ask.call  mid.call  strangle_mult  strangle_prem_sold  \
0      0.160      0.09      0.11     0.100       0.769231            0.184615   
1      0.460      0.32      0.33     0.325       0.254777            0.196178   
2      0.770      0.69      0.71     0.700       0.136054            0.197279   
..       ...       ...       ...       ...            ...                 ...   
765    0.430      0.31      0.33     0.320       0.333333            0.243333   
766    1.520      1.32      1.36     1.340       0.087413            0.246503   
767    2.905      2.67      2.72     2.695       0.044643            0.247768   

     put_mult  put_prem_sold  call_mult  call_prem_sold  
0    1.250000       0.187500   2.000000        0.180000  
1    0.434783       0.195652   0.615385        0.196923  
2    0.259740       0.197403   0.285714        0.197143  
..        ...            ...        ...             ...  
765  0.581395       0.244186   0.781250        0.242187  
766  0.164474       0.246711   0.186567        0.246269  
767  0.086059       0.247849   0.092764        0.247681  

[768 rows x 17 columns]

#### Strangle PNLs

`df_pnl_strangle` contains the strangle trades, along with the strangle scaling factors.

In [9]:
df_pnl_strangle = pd.read_csv("spy_weekly_pnl_strangle_5yr.csv")
df_pnl_strangle

underlying_symbol  underlying_price  type  expiration   data_date  \
0                  SPY        181.610001  call  2013-12-27  2013-12-20   
1                  SPY        182.529999  call  2013-12-27  2013-12-23   
2                  SPY        182.929993  call  2013-12-27  2013-12-24   
...                ...               ...   ...         ...         ...   
8949               SPY        274.359985   put  2018-11-30  2018-11-28   
8950               SPY        273.950012   put  2018-11-30  2018-11-29   
8951               SPY        275.684998   put  2018-11-30  2018-11-30   

      strike   bid   ask    mid  volume      ...           delta      vega  \
0      184.5  0.09  0.11  0.100   463.0      ...        0.098994  0.039879   
1      184.5  0.07  0.08  0.075  1781.0      ...        0.100884  0.035166   
2      184.5  0.07  0.10  0.085  1496.0      ...        0.133438  0.035131   
...      ...   ...   ...    ...     ...      ...             ...       ...   
8949   263.0  0.05  0.06  0.055  2860.0      ...        0.024703  0.014135   
8950   263.0  0.01  0.02  0.015  3717.0      ...        0.009202  0.004279   
8951   263.0  0.00  0.00  0.000    37.0      ...        0.000000  0.000000   

         theta  variation  dly_opt_pnl  dly_dh_pnl  dly_opt_mid_pnl  \
0    -0.046788        0.1        -0.02    0.000000            0.000   
1    -0.046285        0.1         0.03    0.091074            0.025   
2    -0.068293        0.1        -0.02    0.040353           -0.010   
...        ...        ...          ...         ...              ...   
8949 -0.084771        0.5         0.46   -1.019929            0.455   
8950 -0.059818        0.5         0.04    0.010128            0.040   
8951  0.000000        0.5         0.02   -0.015965            0.015   

      dly_tot_pnl  dly_tot_mid_pnl  strangle_mult  
0       -0.020000         0.000000       0.769231  
1        0.121074         0.116074       0.769231  
2        0.020353         0.030353       0.769231  
...           ...              ...            ...  
8949    -0.559929        -0.564929       0.044643  
8950     0.050128         0.050128       0.044643  
8951     0.004035        -0.000965       0.044643  

[8952 rows x 24 columns]

#### Put PNLs

`df_pnl_puts` contains the put trades, along with the put position scaling factors.

In [10]:
df_pnl_put = pd.read_csv("spy_weekly_pnl_put_5yr.csv")
df_pnl_put

underlying_symbol  underlying_price type  expiration   data_date  strike  \
0                  SPY        181.610001  put  2013-12-27  2013-12-20   177.5   
1                  SPY        182.529999  put  2013-12-27  2013-12-23   177.5   
2                  SPY        182.929993  put  2013-12-27  2013-12-24   177.5   
...                ...               ...  ...         ...         ...     ...   
4473               SPY        274.359985  put  2018-11-30  2018-11-28   263.0   
4474               SPY        273.950012  put  2018-11-30  2018-11-29   263.0   
4475               SPY        275.684998  put  2018-11-30  2018-11-30   263.0   

       bid   ask    mid  volume    ...        delta      vega     theta  \
0     0.15  0.17  0.160  3216.0    ...     0.100255  0.040250 -0.073485   
1     0.05  0.06  0.055  2174.0    ...     0.044652  0.018754 -0.046832   
2     0.01  0.04  0.025   340.0    ...     0.023237  0.008964 -0.039275   
...    ...   ...    ...     ...    ...          ...       ...       ...   
4473  0.05  0.06  0.055  2860.0    ...     0.024703  0.014135 -0.084771   
4474  0.01  0.02  0.015  3717.0    ...     0.009202  0.004279 -0.059818   
4475  0.00  0.00  0.000    37.0    ...     0.000000  0.000000  0.000000   

      variation  dly_opt_pnl  dly_dh_pnl  dly_opt_mid_pnl  dly_tot_pnl  \
0           0.1        -0.02    0.000000            0.000    -0.020000   
1           0.1         0.11   -0.092234            0.105     0.017766   
2           0.1         0.02   -0.017860            0.030     0.002140   
...         ...          ...         ...              ...          ...   
4473        0.5         0.46   -1.019929            0.455    -0.559929   
4474        0.5         0.04    0.010128            0.040     0.050128   
4475        0.5         0.02   -0.015965            0.015     0.004035   

      dly_tot_mid_pnl  put_mult  
0            0.000000  1.250000  
1            0.012766  1.250000  
2            0.012140  1.250000  
...               ...       ...  
4473        -0.564929  0.086059  
4474         0.050128  0.086059  
4475        -0.000965  0.086059  

[4476 rows x 24 columns]

#### Call PNLs

`df_pnl_call` contains the call trades, along with the call position scaling factors.

In [11]:
df_pnl_call = pd.read_csv("spy_weekly_pnl_call_5yr.csv")
df_pnl_call

underlying_symbol  underlying_price  type  expiration   data_date  \
0                  SPY        181.610001  call  2013-12-27  2013-12-20   
1                  SPY        182.529999  call  2013-12-27  2013-12-23   
2                  SPY        182.929993  call  2013-12-27  2013-12-24   
...                ...               ...   ...         ...         ...   
4473               SPY        274.359985  call  2018-11-30  2018-11-28   
4474               SPY        273.950012  call  2018-11-30  2018-11-29   
4475               SPY        275.684998  call  2018-11-30  2018-11-30   

      strike        bid        ask        mid  volume    ...         delta  \
0      184.5   0.090000   0.110000   0.100000   463.0    ...      0.098994   
1      184.5   0.070000   0.080000   0.075000  1781.0    ...      0.100884   
2      184.5   0.070000   0.100000   0.085000  1496.0    ...      0.133438   
...      ...        ...        ...        ...     ...    ...           ...   
4473   263.5  10.710000  11.000000  10.855000    41.0    ...      0.950773   
4474   263.5  10.550000  10.830000  10.690000     4.0    ...      0.964959   
4475   263.5  12.184998  12.184998  12.184998     5.0    ...      1.000000   

          vega     theta  variation  dly_opt_pnl  dly_dh_pnl  dly_opt_mid_pnl  \
0     0.039879 -0.046788        0.1    -0.020000    0.000000         0.000000   
1     0.035166 -0.046285        0.1     0.030000    0.091074         0.025000   
2     0.035131 -0.068293        0.1    -0.020000    0.040353        -0.010000   
...        ...       ...        ...          ...         ...              ...   
4473  0.024884 -0.169722        0.5    -5.580000    4.932158        -5.465000   
4474  0.013355 -0.232340        0.5     0.170000   -0.389792         0.165000   
4475  0.000000  0.000000        0.5    -1.354998    1.674189        -1.494998   

      dly_tot_pnl  dly_tot_mid_pnl  call_mult  
0       -0.020000         0.000000   2.000000  
1        0.121074         0.116074   2.000000  
2        0.020353         0.030353   2.000000  
...           ...              ...        ...  
4473    -0.647841        -0.532841   0.092764  
4474    -0.219791        -0.224792   0.092764  
4475     0.319191         0.179191   0.092764  

[4476 rows x 24 columns]

## Demonstration Calculations

In this section we consider some demonstration calculations with the data.

#### Total Premium

In [12]:
df_position_scaling.groupby('variation')['strangle_prem_sold'].sum() * 100000/75 # same position sizing as backtest

variation
0.1    79407.925604
0.3    81311.483403
0.5    81663.062213
Name: strangle_prem_sold, dtype: float64

#### Daily PNLs

In [13]:
df_pnl_strangle['scaled_pnl_dly'] = df_pnl_strangle.dly_tot_pnl * df_pnl_strangle.strangle_mult
df_pnl_put['scaled_pnl_dly'] = df_pnl_put.dly_tot_pnl * df_pnl_put.put_mult
df_pnl_call['scaled_pnl_dly'] = df_pnl_call.dly_tot_pnl * df_pnl_call.call_mult

#### Total Put PNL


In [14]:
df_pnl_put.groupby('variation')['scaled_pnl_dly'].sum() * 100000/75

variation
0.1    22191.239143
0.3     6666.978969
0.5     3698.022828
Name: scaled_pnl_dly, dtype: float64

#### Total Strangle PNL (the 10% variation is what I showed in the 5yr backtest I sent you)

In [15]:
df_pnl_strangle.groupby('variation')['scaled_pnl_dly'].sum() * 100000/75

variation
0.1    22295.014909
0.3     6789.980879
0.5     4263.914719
Name: scaled_pnl_dly, dtype: float64

#### All Strangle Trades Along with Position Sizing Factors (for 10% variation)

In [16]:
pd.\
merge(df_pnl_strangle, df_chain_desc, on='expiration').\
query("data_date == execution & variation == 0.1")

underlying_symbol  underlying_price  type  expiration   data_date  \
0                  SPY        181.610001  call  2013-12-27  2013-12-20   
5                  SPY        181.610001   put  2013-12-27  2013-12-20   
30                 SPY        183.845001  call  2014-01-03  2013-12-27   
...                ...               ...   ...         ...         ...   
8891               SPY        273.730011   put  2018-11-23  2018-11-16   
8916               SPY        263.250000  call  2018-11-30  2018-11-23   
8922               SPY        263.250000   put  2018-11-30  2018-11-23   

      strike   bid   ask    mid   volume       ...         dly_tot_mid_pnl  \
0      184.5  0.09  0.11  0.100    463.0       ...                     0.0   
5      177.5  0.15  0.17  0.160   3216.0       ...                     0.0   
30     186.5  0.09  0.11  0.100   1823.0       ...                     0.0   
...      ...   ...   ...    ...      ...       ...                     ...   
8891   265.0  0.32  0.33  0.325  21018.0       ...                     0.0   
8916   271.0  0.31  0.33  0.320   1638.0       ...                     0.0   
8922   252.0  0.42  0.44  0.430   3012.0       ...                     0.0   

      strangle_mult  scaled_pnl_dly  underlying  monthly  last_trade_date  \
0          0.769231       -0.015385         SPY    False       2013-12-27   
5          0.769231       -0.015385         SPY    False       2013-12-27   
30         0.833333       -0.016667         SPY    False       2014-01-03   
...             ...             ...         ...      ...              ...   
8891       0.370370       -0.003704         SPY    False       2018-11-23   
8916       0.333333       -0.006667         SPY    False       2018-11-30   
8922       0.333333       -0.006667         SPY    False       2018-11-30   

       execution  d2x  num_opts  exec_day_volume  
0     2013-12-20    4        22           516575  
5     2013-12-20    4        22           516575  
30    2013-12-27    4        20           198107  
...          ...  ...       ...              ...  
8891  2018-11-16    4        51           368356  
8916  2018-11-23    5        64           212192  
8922  2018-11-23    5        64           212192  

[512 rows x 32 columns]

## Python Practice

In [18]:
df_position_scaling \
    [['variation', 'strangle_prem_sold']] \
    .groupby('variation') \
    .describe()

strangle_prem_sold                                          \
                       count      mean       std       min       25%   
variation                                                              
0.1                    256.0  0.232640  0.018806  0.181132  0.237447   
0.3                    256.0  0.238217  0.018874  0.192941  0.245243   
0.5                    256.0  0.239247  0.018923  0.194505  0.246778   

                                         
                50%       75%       max  
variation                                
0.1        0.240741  0.243056  0.246032  
0.3        0.246753  0.247423  0.249123  
0.5        0.247844  0.248319  0.249224

In [19]:
df_position_scaling.columns

Index(['variation', 'underlying', 'expiration', 'execution', 'd2x', 'bid.put',
       'ask.put', 'mid.put', 'bid.call', 'ask.call', 'mid.call',
       'strangle_mult', 'strangle_prem_sold', 'put_mult', 'put_prem_sold',
       'call_mult', 'call_prem_sold'],
      dtype='object')